# TEMPLATE JAWABAN TUGAS LANGUAGE MODELING

### Import Library

In [41]:
import nltk
import xlrd
import operator
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Import Dataset (Training dan Testing)

In [3]:
book = xlrd.open_workbook('dataTrainTitle.xlsx')
sheet = book.sheet_by_name('Sheet1')
dataTrain = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]
print(dataTrain)

[["It'll Be Years Before The New 'R36' Nissan GT-R Arrives"], ['A New, Potentially Less Ugly Nissan Juke Is On The Way'], ['The 2020 Nissan GT-R Nismo Is Almost 600LT Money']]


### Bangun Model Unigram

In [51]:
def buildUnigramModel(trainingSentences):
    wordCount = {}
    count = 0
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence[0])
        for token in tokens:
            tokenLower = token.lower()
            if tokenLower in wordCount:
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
            count += 1
    unigramModel = wordCount
    #count prob
    for word in unigramModel:
        unigramModel[word] = unigramModel[word] / count
    return unigramModel
unigramModel = buildUnigramModel(dataTrain)

### Bangun Model Bigram

In [50]:
def buildBigramModel(trainingSentences):
    wordPairCount = {}
    wordCount = {}
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    wordPairCount['<s>'] = {}
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence[0])
        counter = 0
        tokenBefore = ''
        wordCount['<s>'] += 1
        wordCount['</s>'] += 1
        for token in tokens:
            tokenLower = token.lower()
            counter += 1
            if tokenLower not in wordPairCount:
                wordPairCount[tokenLower] = {}
            if counter == 1:
                if tokenLower in wordPairCount['<s>']:
                    wordPairCount['<s>'][tokenLower] += 1
                else:
                    wordPairCount['<s>'][tokenLower] = 1
            elif counter == len(tokens):
                if tokenLower in wordPairCount[tokenBefore]:
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
                if '</s>' in wordPairCount[tokenLower]:
                    wordPairCount[tokenLower]['</s>'] += 1
                else:
                    wordPairCount[tokenLower]['</s>'] = 1
            else:
                if tokenLower in wordPairCount[tokenBefore]:
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
            tokenBefore = tokenLower
            if tokenLower in wordCount:
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
    bigramModel = {}
    vee = len(wordCount) #for smoothing
    #smoothing (adding one)
    for wordBefore in wordPairCount.keys():
        for theWord in wordCount.keys():
            if theWord != '<s>':
                if theWord not in wordPairCount[wordBefore]:
                    wordPairCount[wordBefore][theWord] = 1
                else:
                    wordPairCount[wordBefore][theWord] += 1
                #bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    #smoothing (adding vee)
    for theWord in wordCount.keys():
        wordCount[theWord] += vee
    #count prob
    for wordBefore in wordPairCount.keys():
        bigramModel[wordBefore] = {}
        for theWord in wordCount.keys():
            if theWord != '<s>':
                bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    return bigramModel
bigramModel = buildBigramModel(dataTrain)

### Bangun Fungsi Kata Berikutnya Dengan Probability Tertinggi

In [53]:
#Diberikan sebuah kata, cari kata berikutnya yang memiliki probability tertinggi berdasarkan model bigram
def nextBestWord(bigramModel, currentWord):
    if currentWord not in bigramModel:
        nextWord = "OOV"
    else:
        nextWord = max(bigramModel[currentWord].items(), key=operator.itemgetter(1))[0]
    return nextWord
#Lakukan percobaan dengan memberikan sebuah kata, lalu print kata berikutnya
print(nextBestWord(bigramModel,"nissan"))

gt-r


### Hitung Perplexity Unigram dan Bigram terhadap Data Testing

In [ ]:
def countPerplexity(..........., testSentences):
    

    
    return perplexity

### Bangun Fungsi Generate Kalimat

In [ ]:
def generateSentence(......, length):
    
    
    
    return sentence